In [1]:
import json

from matplotlib import pyplot as plt
import numpy as np

In [2]:
np.set_printoptions(edgeitems=6, linewidth=100, 
    formatter=dict(float=lambda x: "%.4g" % x))

In [3]:
import sys

sys.path.append("/home/papjuli/code/ETH/3DV_project/CityMagic3D")

In [4]:
from processing import BlocksExchange_xml_parser

In [5]:
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [13]:
from instance_masks_from_images.utils import get_extrinsic_matrix

In [6]:
caminfo_path = "/media/papjuli/Data/data/3DV_project/STPLS3D/Source_Images/Real_World/WMSC_CamInfoCC.xml"


In [7]:
intrinsic_matrix, poses_for_images, width, height = BlocksExchange_xml_parser.parse_xml(caminfo_path)

In [8]:
intrinsic_matrix

array([[3633, 0, 2437],
       [0, 3633, 1838],
       [0, 0, 1]])

In [9]:
from instance_masks_from_images.scene import Camera, Scene

In [10]:
camera = Camera(intrinsic_matrix, width, height)

In [11]:
resolution = (width, height)
resolution

(4864, 3648)

In [12]:
image_name = "DJI_0859.JPG"

rot, pos = poses_for_images[image_name]

In [14]:
extrinsic = get_extrinsic_matrix(rot, pos)
extrinsic

array([[0.4274, -0.904, -0.01106, -30.31],
       [-0.6441, -0.2959, -0.7054, 55.03],
       [0.6344, 0.3086, -0.7088, -71.84],
       [0, 0, 0, 1]])

In [15]:

camera_lines = o3d.geometry.LineSet.create_camera_visualization(
    view_width_px=resolution[0], view_height_px=resolution[1], 
    intrinsic=camera.intrinsic_matrix.copy(), 
    extrinsic=extrinsic,
    scale=20)

In [16]:
marker = o3d.geometry.TriangleMesh.create_tetrahedron()
marker.translate(pos)
marker.paint_uniform_color([1, 0, 0])

TriangleMesh with 4 points and 4 triangles.

In [17]:
class Config:
    def __init__(self, point_cloud_path, cam_info_path, images_dir, mesh_path, visibility_threshold):
        self.point_cloud_path = point_cloud_path
        self.cam_info_path = cam_info_path
        self.images_dir = images_dir
        self.mesh_path = mesh_path
        self.visibility_threshold = visibility_threshold

cfg = {
    "point_cloud_path": "/media/papjuli/Data/data/3DV_project/STPLS3D/point_clouds/RealWorldData/WMSC_points.ply",
    "cam_info_path": "/media/papjuli/Data/data/3DV_project/STPLS3D/Source_Images/Real_World/WMSC_CamInfoCC.xml",
    "images_dir": "/media/papjuli/Data/data/3DV_project/STPLS3D/Source_Images/Real_World/WMSC",
    "mesh_path": "/media/papjuli/Data/data/3DV_project/processed/STPLS3D/mesh/WMSC.ply",
    "visibility_threshold": 0.98
}
cfg = Config(**cfg)

scene = Scene(cfg)

In [18]:
coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=20.0)

o3d.visualization.draw_geometries([scene.point_cloud, camera_lines, coord_frame, marker],
                                  zoom=0.34,
                                  front=[1, 0, 0.5],
                                  lookat=[0, 0, 0],
                                  up=[-0.5, 0, 1])

In [22]:
mask_indices_path = "/home/papjuli/code/ETH/3DV_project/CityMagic3D/outputs/instance_masks_from_images/2024-05-14_21-04-28/DJI_0027.JPG__mask_indices.npz"

mask_indices = np.load(mask_indices_path)
mask_indices_ls = [mask_indices[k] for k in mask_indices]

for mask in mask_indices_ls:
    np.asarray(scene.point_cloud.colors)[mask,:] = (np.random.random(3) + np.asarray(scene.point_cloud.colors)[mask,:]) / 2



In [23]:

o3d.visualization.draw_geometries([scene.point_cloud, camera_lines, coord_frame],
                                  zoom=0.34,
                                  front=[1, 0, 0.5],
                                  lookat=[0, 0, 0],
                                  up=[-0.5, 0, 1])